In [159]:
# system dependecies


# library dependeceis
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

# online
import duckdb
import requests


In [160]:
import os
import scipy.stats
import sklearn.preprocessing
import sklearn.model_selection
import sklearn.neighbors
import duckdb

from Bio import pairwise2
from Bio.Seq import Seq
import requests

In [161]:
# seaborn plot configurations
sns.set_context('talk')
sns.set_style('whitegrid')
palette = sns.color_palette('colorblind')

In [162]:
# CSV reading
#, index_col=0
df = pd.read_csv("/Users/amin/ValidProt/data/Sample.csv")
df

,Unnamed: 0,prot_pair_index,meso_seq,thermo_seq,meso_ogt,thermo_ogt,scaled_local_symmetric_percent_id,local_E_value,scaled_local_query_percent_id,local_gap_compressed_percent_id
0,1256842,126227630,MLLSDRDLVSEIKSGDLSLEPFEPALLQPSSIDVRLDRFFRVFNNH...,MLLSDRDLRKELESGRLELDPFDPAMLQPSSIDVRLDRFFRVFDNT...,27.5,45.0,0.777202,0.0,0.773196,0.802139
1,1456567,169784592,MRFEGTSGYVATDDLKVAVNAAIALERPLLVKGEPGTGKTVLAVEV...,MKFTGSDSYVATEDLMIAVNAAVTLERPLLVKGEPGTGKTELARQV...,30.0,54.0,0.782143,0.0,0.784946,0.784946
2,874464,31933768,MAYETINVDVQDHVCLIKLHRPEALNALNAALVSELCTALEEADAS...,MAYKTIIVEIEDHVALIKLNRPEALNALNSELLGELAQAVTEADAN...,19.5,54.0,0.775194,0.0,0.775194,0.775194
3,560201,32409414,MAIRKYKPTTPGRRGSSVADFAEITRSTPEKSLLRPLSKTGGRNNQ...,MGIRKYKPTTPGRRGASVADFVELTRREPEKSLLRPLPKKGGRNNR...,28.0,52.5,0.787770,0.0,0.790614,0.802198
4,33257,175862226,MLQRLQDRVAVVTGGGSGIGLATVRRFAAEGAKVVVADIDAAAGEA...,MSEDIICRRLTGRTAVVTGAGSGIGLASARRLASEGANVVCADVDE...,28.0,45.0,0.788350,0.0,0.780769,0.802372
...,...,...,...,...,...,...,...,...,...,...
9995,458544,42061208,MRFVIARCQVDYVGRLTAHLPMANRLVMVKSDGSVLVHSDGGSYKP...,MRLVIARCQVDYVGRLTAHLPMAQRLLLIKADGSVSVHSDDRAYKP...,28.0,45.0,0.872146,0.0,0.872146,0.872146
9996,469612,165123678,MKPIVGSIVALITPMHEDGSVDYPALRKLIDWHIAEGTDCIGVVGT...,MTSSRVTLTGSIVALVTPMHEDGSVDYPTLRKLIDWHIAQGTDCIS...,30.0,48.0,0.764805,0.0,0.758389,0.776632
9997,226964,71308777,MSFFAPKTVVSAHCDLPCGVYDPAQARIEAESIKAVAEKYQANTDP...,MLSRLFAPTVEVSAHCDLPCGVYDPAQARIEAQSIKAIIEKYHASD...,30.0,52.5,0.777358,0.0,0.768657,0.804688
9998,123183,137720058,MRLVIARCSVDYVGRLTAHLPMATRLLLVKADGSVSVHADDRAYKP...,MRLVIARCQVDYHGRLTAHLPMATRLVLIKADGSVSIHSDDRAYKP...,32.0,50.0,0.737557,0.0,0.744292,0.740909


# Pairing

In [163]:
df["meso_seq"].iloc[0]+df["thermo_seq"].iloc[0]

'MLLSDRDLVSEIKSGDLSLEPFEPALLQPSSIDVRLDRFFRVFNNHLYTHIDPAEQQDDLTAEVEVTDGEAFVLHPGEFVLASTLEVITLGDQLAGRLEGKSSLGRLGLLTHSTAGFIDPGFSGHVTLELSNVANLPIKLWPGMKIGQLCIFRLSSPAEHPYGSAVYGSRYQGQRGPTPSRSAQNFRLWPTSMLLSDRDLRKELESGRLELDPFDPAMLQPSSIDVRLDRFFRVFDNTKYTHIDPSIQQDELTSLVEVGDGDEPFVLHPGEFVLGSTYEMVTLPDDLAGRLEGKSSLGRLGLLTHSTAGFIDPGFSGHITLELSNVANLPITLWPGMKIGQLCLFRLTSAAEHPYGSARAGSRYQHQRGPTPSRAYLNFSRIDTRR'

# We need that in the future

In [164]:
from Bio.Seq import Seq

seq_list = []
for i in range(min(len(df['meso_seq']), len(df['thermo_seq']))):
    seq1 = Seq(df['meso_seq'][i])
    seq2 = Seq(df['thermo_seq'][i])
    seq12 = seq1 + seq2
    seq_list.append((seq1, seq2, seq12))
samples = pd.DataFrame(seq_list, columns=['meso_seq', 'thermo_seq', 'seq12'])
print(samples)

                                               meso_seq  \
0     (M, L, L, S, D, R, D, L, V, S, E, I, K, S, G, ...   
1     (M, R, F, E, G, T, S, G, Y, V, A, T, D, D, L, ...   
2     (M, A, Y, E, T, I, N, V, D, V, Q, D, H, V, C, ...   
3     (M, A, I, R, K, Y, K, P, T, T, P, G, R, R, G, ...   
4     (M, L, Q, R, L, Q, D, R, V, A, V, V, T, G, G, ...   
...                                                 ...   
9995  (M, R, F, V, I, A, R, C, Q, V, D, Y, V, G, R, ...   
9996  (M, K, P, I, V, G, S, I, V, A, L, I, T, P, M, ...   
9997  (M, S, F, F, A, P, K, T, V, V, S, A, H, C, D, ...   
9998  (M, R, L, V, I, A, R, C, S, V, D, Y, V, G, R, ...   
9999  (M, T, S, P, K, Q, Q, Q, C, G, F, V, A, V, I, ...   

                                             thermo_seq  \
0     (M, L, L, S, D, R, D, L, R, K, E, L, E, S, G, ...   
1     (M, K, F, T, G, S, D, S, Y, V, A, T, E, D, L, ...   
2     (M, A, Y, K, T, I, I, V, E, I, E, D, H, V, A, ...   
3     (M, G, I, R, K, Y, K, P, T, T, P, G, R, R, G, ...

# Alignment (we need that in the future)

In [166]:
# Creating sample sequences
seqpair = []
for i in range(0,10000):
    seq1 = Seq(df['meso_seq'][i])
    seq2 = Seq(df['thermo_seq'][i])
    seqpair.append((seq1, seq2))

# Finding similarities
results = []
for seq1, seq2 in seqpair:
    alignments = pairwise2.align.globalxx(seq1, seq2)

    # Adding results to a list
    for match in alignments:
        results.append({
            'seqA': match.seqA,
            'seqB': match.seqB,
            'score': match.score,
            'start': match.start,
            'end': match.end
        })

# Creating a pandas DataFrame
alignment = pd.DataFrame(results)

# Printing the DataFrame
alignment


,seqA,seqB,score,start,end
0,MLLSDRDLVSEI-KSGD-L-S--LE--PFE-PAL-LQPSSIDVRLD...,MLLSDRDL----RK---ELESGRLELDPF-DPA-MLQPSSIDVRLD...,154.0,0,232
1,MLLSDRDLVSEIKSGD-L-S--LE--PFE-PAL-LQPSSIDVRLDR...,MLLSDRDL---RK---ELESGRLELDPF-DPA-MLQPSSIDVRLDR...,154.0,0,231
2,MLLSDRDLVSEIKSGD-L-S--LE--PFE-PAL-LQPSSIDVRLDR...,MLLSDRDL--R-K---ELESGRLELDPF-DPA-MLQPSSIDVRLDR...,154.0,0,231
3,MLLSDRDLVSEIKSGD-L-S--LE--PFE-PAL-LQPSSIDVRLDR...,MLLSDRDL-R--K---ELESGRLELDPF-DPA-MLQPSSIDVRLDR...,154.0,0,231
4,MLLSDRDLVSEIKSGD-L-S--LE--PFE-PAL-LQPSSIDVRLDR...,MLLSDRDLR---K---ELESGRLELDPF-DPA-MLQPSSIDVRLDR...,154.0,0,231
...,...,...,...,...,...
3529514,MTSPKQQQCGFVAVI-GAPNAGKSTLVN-AMVGQKVAIVSAKAQTT...,MT-EK---CGLVAV-LGAPNAGKSTLVNTL-VGQKVAIVSAKAQTT...,227.0,0,367
3529515,MTSPKQQQCGFVAVI-GAPNAGKSTLVN-AMVGQKVAIVSAKAQTT...,MTE-K---CGLVAV-LGAPNAGKSTLVNTL-VGQKVAIVSAKAQTT...,227.0,0,367
3529516,MTSP-KQQQCGF-VAVIGAPNAGKSTLVN-AMVGQKVAIVSAKAQT...,MT--EK---CG-LVAVLGAPNAGKSTLVNTL-VGQKVAIVSAKAQT...,227.0,0,368
3529517,MTSPKQQQCGF-VAVIGAPNAGKSTLVN-AMVGQKVAIVSAKAQTT...,MT-EK---CG-LVAVLGAPNAGKSTLVNTL-VGQKVAIVSAKAQTT...,227.0,0,367


# API: with protein ID

In [100]:
import requests

accession = "IPR002394"
url = f"https://www.ebi.ac.uk/interpro/api/entry/interpro/{accession}"
response = requests.get(url)

if response.ok:
    data2 = response.json()
    # process the data as needed
else:
    print("Request failed")

    
print(data2)

{'metadata': {'accession': 'IPR002394', 'entry_id': None, 'type': 'family', 'go_terms': [{'identifier': 'GO:0005216', 'name': 'ion channel activity', 'category': {'code': 'F', 'name': 'molecular_function'}}, {'identifier': 'GO:0022848', 'name': 'acetylcholine-gated cation-selective channel activity', 'category': {'code': 'F', 'name': 'molecular_function'}}, {'identifier': 'GO:0006811', 'name': 'ion transport', 'category': {'code': 'P', 'name': 'biological_process'}}, {'identifier': 'GO:0016020', 'name': 'membrane', 'category': {'code': 'C', 'name': 'cellular_component'}}, {'identifier': 'GO:0045211', 'name': 'postsynaptic membrane', 'category': {'code': 'C', 'name': 'cellular_component'}}], 'source_database': 'interpro', 'member_databases': {'prints': {'PR00254': 'NICOTINICR'}}, 'integrated': None, 'hierarchy': {'accession': 'IPR006201', 'name': 'Neurotransmitter-gated ion-channel', 'type': 'Family', 'children': [{'accession': 'IPR002394', 'name': 'Nicotinic acetylcholine receptor', 't

# API: by sequence (Families: `accession`, `start`, `end`, `score`, `dc-status`)

In [121]:
# with /entry/pfam :) good jub - Here we have access to the entry/Pfam data 
sequence = df["meso_seq"].iloc[0]+df["thermo_seq"].iloc[0]
url2 = f"https://www.ebi.ac.uk:443/interpro/api/protein/reviewed/Q6ZNL6/entry/pfam?sequence={sequence}"
response2 = requests.get(url2)
response2.json()

{'metadata': {'accession': 'Q6ZNL6',
  'id': 'FGD5_HUMAN',
  'source_organism': {'taxId': '9606',
   'scientificName': 'Homo sapiens',
   'fullName': 'Homo sapiens (Human)'},
  'name': 'FYVE, RhoGEF and PH domain-containing protein 5',
  'description': ['Activates CDC42, a member of the Ras-like family of Rho- and Rac proteins, by exchanging bound GDP for free GTP. Mediates VEGF-induced CDC42 activation. May regulate proangiogenic action of VEGF in vascular endothelial cells, including network formation, directional movement and proliferation. May play a role in regulating the actin cytoskeleton and cell shape'],
  'length': 1462,
  'sequence': 'MFRGPKPPIAPKPRLTAPNEWRASVYLNDSLNKCSNGRLPCVDRGLDEGPRSIPKCSESETDEDYIVVPRVPLREDEPKDEGSVGNKALVSPESSAEEEEEREEGGEACGLEGTGAGEDSVAPAAPGAGALSREGEEGTDLALEDEGEGCADEPGTLEQVSRSEEEEKLVQPHRECSLEDSGPWAGEGVFQSDLLLPHIHGEDQEPPDTPGEAEEDDEEGCASTDPAGADEGSGPDRPTEDMGQDAEDTSEEPPEKEELAGVQEAETATDCPEVLEEGCEEATGVTGGEQVDLSEPPDHEKKTNQEVAAATLEDHAQDESAEESCQIVPFENDCMEDFVTSLTGSP

In [178]:
# or
# with /entry/pfam :) good jub - Here we have access to the entry/Pfam data 
sequence = df["meso_seq"].iloc[0]+df["thermo_seq"].iloc[0]
url2 = f"https://www.ebi.ac.uk:443/interpro/api/protein/reviewed/entry/pfam?sequence={sequence}"
response2 = requests.get(url2)
response2.json()

{'count': 537433,
 'next': 'https://www.ebi.ac.uk/interpro/api/protein/reviewed/entry/pfam?cursor=source%3As%3Aa0a023g9n9&sequence=MLLSDRDLVSEIKSGDLSLEPFEPALLQPSSIDVRLDRFFRVFNNHLYTHIDPAEQQDDLTAEVEVTDGEAFVLHPGEFVLASTLEVITLGDQLAGRLEGKSSLGRLGLLTHSTAGFIDPGFSGHVTLELSNVANLPIKLWPGMKIGQLCIFRLSSPAEHPYGSAVYGSRYQGQRGPTPSRSAQNFRLWPTSMLLSDRDLRKELESGRLELDPFDPAMLQPSSIDVRLDRFFRVFDNTKYTHIDPSIQQDELTSLVEVGDGDEPFVLHPGEFVLGSTYEMVTLPDDLAGRLEGKSSLGRLGLLTHSTAGFIDPGFSGHITLELSNVANLPITLWPGMKIGQLCLFRLTSAAEHPYGSARAGSRYQHQRGPTPSRAYLNFSRIDTRR',
 'previous': None,
 'results': [{'metadata': {'accession': 'A0A009IHW8',
    'name': 'NAD(+) hydrolase AbTIR',
    'source_database': 'reviewed',
    'length': 269,
    'source_organism': {'taxId': '1310613',
     'scientificName': 'Acinetobacter baumannii (strain 1295743)',
     'fullName': 'Acinetobacter baumannii (strain 1295743)'}},
   'entry_subset': [{'accession': 'PF13676',
     'entry_protein_locations': [{'fragments': [{'start': 138,
         'end': 249,
         'dc

# Put the results in table :)

In [195]:
import pandas as pd
import requests

# define endpoint and parameters
endpoint = 'https://www.ebi.ac.uk/interpro/api/protein/reviewed/entry/pfam'
params = {'query': 'df["meso_seq"].iloc[0]+df["thermo_seq"].iloc[0]'}

# define function to get all results
def get_all_results():
    all_results = []
    while True:
        # make request
        response = requests.get(endpoint, params=params)
        # check if there are no more results
        if not response.json().get('results'):
            break
        # add results to list
        all_results.extend(response.json().get('results'))
        # update cursor parameter to get next page
        params['cursor'] = response.json().get('next').split('&')[1]
    return all_results

# call function to get all results
results = get_all_results()

# create table from results
table = pd.DataFrame()
for result in results:
    metadata = result['metadata']
    accession = metadata['accession']
    name = metadata['name']
    source_database = metadata['source_database']
    length = metadata['length']
    tax_id = metadata['source_organism']['taxId']
    scientific_name = metadata['source_organism']['scientificName']
    full_name = metadata['source_organism']['fullName']
    for entry_subset in result['entry_subset']:
        accession_subset = entry_subset['accession']
        protein_length = entry_subset['protein_length']
        source_database_subset = entry_subset['source_database']
        entry_type = entry_subset['entry_type']
        entry_integrated = entry_subset['entry_integrated']
        for location in entry_subset['entry_protein_locations']:
            start = location['fragments'][0]['start']
            end = location['fragments'][0]['end']
            dc_status = location['fragments'][0]['dc-status']
            model = location['model']
            score = location['score']
            row = {'accession': accession,
                   'name': name,
                   'source_database': source_database,
                   'length': length,
                   'tax_id': tax_id,
                   'scientific_name': scientific_name,
                   'full_name': full_name,
                   'accession_subset': accession_subset,
                   'protein_length': protein_length,
                   'source_database_subset': source_database_subset,
                   'entry_type': entry_type,
                   'entry_integrated': entry_integrated,
                   'start': start,
                   'end': end,
                   'dc_status': dc_status,
                   'model': model,
                   'score': score}
            table = table.append(row, ignore_index=True)

# print table
table


/var/folders/t9/1bs6fqy15y113y80dn0vrrb00000gn/T/ipykernel_84205/1798144793.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(row, ignore_index=True)
/var/folders/t9/1bs6fqy15y113y80dn0vrrb00000gn/T/ipykernel_84205/1798144793.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(row, ignore_index=True)
/var/folders/t9/1bs6fqy15y113y80dn0vrrb00000gn/T/ipykernel_84205/1798144793.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(row, ignore_index=True)
/var/folders/t9/1bs6fqy15y113y80dn0vrrb00000gn/T/ipykernel_84205/1798144793.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

,accession,name,source_database,length,tax_id,scientific_name,full_name,accession_subset,protein_length,source_database_subset,entry_type,entry_integrated,start,end,dc_status,model,score
0,A0A009IHW8,NAD(+) hydrolase AbTIR,reviewed,269,1310613,Acinetobacter baumannii (strain 1295743),Acinetobacter baumannii (strain 1295743),PF13676,269,pfam,domain,ipr000157,138,249,CONTINUOUS,PF13676,5.400000e-20
1,A0A011QK89,L-2-hydroxyglutarate dehydrogenase,reviewed,388,522306,Accumulibacter regalis,Accumulibacter regalis,PF01266,388,pfam,domain,ipr006076,6,374,CONTINUOUS,PF01266,2.400000e-66
2,A0A017SE81,Short-chain dehydrogenase fogD,reviewed,286,1388766,Aspergillus ruber (strain CBS 135680),Aspergillus ruber (strain CBS 135680),PF00106,286,pfam,domain,ipr002347,4,189,CONTINUOUS,PF00106,1.600000e-38
3,A0A017SE85,Transcriptional regulator fogI,reviewed,415,1388766,Aspergillus ruber (strain CBS 135680),Aspergillus ruber (strain CBS 135680),PF00172,415,pfam,domain,ipr001138,11,46,CONTINUOUS,PF00172,6.000000e-07
4,A0A017SEF3,Prenyltransferase fogH,reviewed,434,1388766,Aspergillus ruber (strain CBS 135680),Aspergillus ruber (strain CBS 135680),PF11991,434,pfam,family,ipr017795,22,381,CONTINUOUS,PF11991,2.000000e-90
5,A0A017SEX7,Highly reducing polyketide synthase fogrA,reviewed,2403,1388766,Aspergillus ruber (strain CBS 135680),Aspergillus ruber (strain CBS 135680),PF00109,2403,pfam,domain,ipr014030,7,254,CONTINUOUS,PF00109,2.700000e-62
6,A0A017SEX7,Highly reducing polyketide synthase fogrA,reviewed,2403,1388766,Aspergillus ruber (strain CBS 135680),Aspergillus ruber (strain CBS 135680),PF00698,2403,pfam,domain,ipr014043,537,837,CONTINUOUS,PF00698,9.800000e-51
7,A0A017SEX7,Highly reducing polyketide synthase fogrA,reviewed,2403,1388766,Aspergillus ruber (strain CBS 135680),Aspergillus ruber (strain CBS 135680),PF02801,2403,pfam,domain,ipr014031,263,377,CONTINUOUS,PF02801,1.900000e-34
8,A0A017SEX7,Highly reducing polyketide synthase fogrA,reviewed,2403,1388766,Aspergillus ruber (strain CBS 135680),Aspergillus ruber (strain CBS 135680),PF08240,2403,pfam,domain,ipr013154,1685,1741,CONTINUOUS,PF08240,9.400000e-06
9,A0A017SEX7,Highly reducing polyketide synthase fogrA,reviewed,2403,1388766,Aspergillus ruber (strain CBS 135680),Aspergillus ruber (strain CBS 135680),PF08659,2403,pfam,family,ipr013968,2006,2185,CONTINUOUS,PF08659,2.000000e-56


# Sort table by 'score' column

In [196]:
# sort table by 'score' column
table = table.sort_values('score', ascending=False)

table

,accession,name,source_database,length,tax_id,scientific_name,full_name,accession_subset,protein_length,source_database_subset,entry_type,entry_integrated,start,end,dc_status,model,score
8,A0A017SEX7,Highly reducing polyketide synthase fogrA,reviewed,2403,1388766,Aspergillus ruber (strain CBS 135680),Aspergillus ruber (strain CBS 135680),PF08240,2403,pfam,domain,ipr013154,1685,1741,CONTINUOUS,PF08240,9.400000e-06
3,A0A017SE85,Transcriptional regulator fogI,reviewed,415,1388766,Aspergillus ruber (strain CBS 135680),Aspergillus ruber (strain CBS 135680),PF00172,415,pfam,domain,ipr001138,11,46,CONTINUOUS,PF00172,6.000000e-07
16,A0A017SGC7,FAD-linked oxidoreductase fogF,reviewed,497,1388766,Aspergillus ruber (strain CBS 135680),Aspergillus ruber (strain CBS 135680),PF08031,497,pfam,domain,ipr012951,448,489,CONTINUOUS,PF08031,7.900000e-09
21,A0A017SQ41,Nonribosomal peptide synthetase echPS,reviewed,2092,1388766,Aspergillus ruber (strain CBS 135680),Aspergillus ruber (strain CBS 135680),PF00550,2092,pfam,domain,ipr009081,1553,1617,CONTINUOUS,PF00550,1.900000e-09
29,A0A023FT45,Evasin P985,reviewed,132,251391,Amblyomma parvum,Amblyomma parvum (South American tick),PF19429,132,pfam,family,ipr045797,48,123,CONTINUOUS,PF19429,3.200000e-11
28,A0A023FFD0,Evasin P991,reviewed,136,34607,Amblyomma cajennense,Amblyomma cajennense (Cayenne tick),PF19429,136,pfam,family,ipr045797,55,128,CONTINUOUS,PF19429,5.800000e-13
10,A0A017SEX7,Highly reducing polyketide synthase fogrA,reviewed,2403,1388766,Aspergillus ruber (strain CBS 135680),Aspergillus ruber (strain CBS 135680),PF13602,2403,pfam,domain,None,1842,1981,CONTINUOUS,PF13602,1.200000e-14
27,A0A023FFB5,Evasin P983,reviewed,119,34607,Amblyomma cajennense,Amblyomma cajennense (Cayenne tick),PF19429,119,pfam,family,ipr045797,37,111,CONTINUOUS,PF19429,1.400000e-15
31,A0A023G9N9,Evasin P1183,reviewed,115,251400,Amblyomma triste,Amblyomma triste (Neotropical tick),PF19429,115,pfam,family,ipr045797,38,108,CONTINUOUS,PF19429,4.000000e-16
25,A0A023FBW7,Evasin P546,reviewed,118,34607,Amblyomma cajennense,Amblyomma cajennense (Cayenne tick),PF19429,118,pfam,family,ipr045797,38,110,CONTINUOUS,PF19429,5.800000e-17


### API: by sequence

In [122]:

import requests

# define the sequences to pair
meso_seq = "MLLSDRDLVSEIKSGDLSLEPFEPALLQPSSIDVRLDRFFRVFNNHLYTHIDPAEQQDDLTAEVEVTDGEAFVLHPGEFVLASTLEVITLGDQLAGRLEGKSSLGRLGLLTHSTAGFIDPGFSGHVTLELSNVANLPIKLWPGMKIGQLCIFRLSSPAEHPYGSAVYGSRYQGQRGPTPSRSAQNFRLWPTS"
thermo_seq = "MLLSDRDLRKELESGRLELDPFDPAMLQPSSIDVRLDRFFRVFDNTKYTHIDPSIQQDELTSLVEVGDGDEPFVLHPGEFVLGSTYEMVTLPDDLAGRLEGKSSLGRLGLLTHSTAGFIDPGFSGHITLELSNVANLPITLWPGMKIGQLCLFRLTSAAEHPYGSARAGSRYQHQRGPTPSRAYLNFSRIDTRR"

# make API requests to retrieve Pfam domain annotations for the sequences
meso_response = requests.get(f"https://www.ebi.ac.uk:443/interpro/api/entry/pfam?size=200&sequence={meso_seq}")
thermo_response = requests.get(f"https://www.ebi.ac.uk:443/interpro/api/entry/pfam?size=200&sequence={thermo_seq}")

# parse the response JSON to extract the Pfam domains
meso_domains = set(hit['metadata']['accession'] for hit in meso_response.json()['results'])
thermo_domains = set(hit['metadata']['accession'] for hit in thermo_response.json()['results'])
print(meso_domains)
# find the common domains
common_domains = meso_domains.intersection(thermo_domains)

# print the results
if common_domains:
    print(f"The common Pfam domains for the two proteins are: {', '.join(common_domains)}")
else:
    print("There are no common Pfam domains for the two proteins.")

{'PF00001', 'PF00005', 'PF00007', 'PF00017', 'PF00019', 'PF00009', 'PF00002', 'PF00015', 'PF00008', 'PF00011', 'PF00010', 'PF00012', 'PF00004', 'PF00016', 'PF00020', 'PF00006', 'PF00014', 'PF00018', 'PF00013', 'PF00003'}
The common Pfam domains for the two proteins are: PF00001, PF00005, PF00007, PF00017, PF00019, PF00009, PF00002, PF00015, PF00008, PF00011, PF00010, PF00012, PF00004, PF00016, PF00020, PF00006, PF00014, PF00018, PF00013, PF00003


### API: by sequence

In [29]:
sequence = df["meso_seq"].iloc[0]
url = f"https://www.ebi.ac.uk:443/interpro/api/protein/reviewed?sequence={sequence}"
response = requests.get(url)
response.json()

{'count': 568744,
 'next': 'https://www.ebi.ac.uk/interpro/api/protein/reviewed?cursor=cD1BMEEwMjNGVDQ1&sequence=MLLSDRDLVSEIKSGDLSLEPFEPALLQPSSIDVRLDRFFRVFNNHLYTHIDPAEQQDDLTAEVEVTDGEAFVLHPGEFVLASTLEVITLGDQLAGRLEGKSSLGRLGLLTHSTAGFIDPGFSGHVTLELSNVANLPIKLWPGMKIGQLCIFRLSSPAEHPYGSAVYGSRYQGQRGPTPSRSAQNFRLWPTS',
 'previous': None,
 'results': [{'metadata': {'accession': 'A0A009IHW8',
    'name': 'NAD(+) hydrolase AbTIR',
    'source_database': 'reviewed',
    'length': 269,
    'source_organism': {'taxId': '1310613',
     'scientificName': 'Acinetobacter baumannii (strain 1295743)',
     'fullName': 'Acinetobacter baumannii (strain 1295743)'}}},
  {'metadata': {'accession': 'A0A011QK89',
    'name': 'L-2-hydroxyglutarate dehydrogenase',
    'source_database': 'reviewed',
    'length': 388,
    'source_organism': {'taxId': '522306',
     'scientificName': 'Accumulibacter regalis',
     'fullName': 'Accumulibacter regalis'}}},
  {'metadata': {'accession': 'A0A017SE81',
    'name': 'Short-chain

### API: by sequence

In [101]:
sequence = df["meso_seq"].iloc[0]
url2 = f"https://www.ebi.ac.uk:443/interpro/api/entry/pfam?sequence={sequence}"
response = requests.get(url)
response.json()

{'metadata': {'accession': 'IPR002394',
  'entry_id': None,
  'type': 'family',
  'go_terms': [{'identifier': 'GO:0005216',
    'name': 'ion channel activity',
    'category': {'code': 'F', 'name': 'molecular_function'}},
   {'identifier': 'GO:0022848',
    'name': 'acetylcholine-gated cation-selective channel activity',
    'category': {'code': 'F', 'name': 'molecular_function'}},
   {'identifier': 'GO:0006811',
    'name': 'ion transport',
    'category': {'code': 'P', 'name': 'biological_process'}},
   {'identifier': 'GO:0016020',
    'name': 'membrane',
    'category': {'code': 'C', 'name': 'cellular_component'}},
   {'identifier': 'GO:0045211',
    'name': 'postsynaptic membrane',
    'category': {'code': 'C', 'name': 'cellular_component'}}],
  'source_database': 'interpro',
  'member_databases': {'prints': {'PR00254': 'NICOTINICR'}},
  'integrated': None,
  'hierarchy': {'accession': 'IPR006201',
   'name': 'Neurotransmitter-gated ion-channel',
   'type': 'Family',
   'children':

In [ ]:
&annotation=alignment

In [ ]:
#!/usr/bin/env python3

# standard library modules
import sys, errno, re, json, ssl
from urllib import request
from urllib.error import HTTPError
from time import sleep

sequence = df["meso_seq"].iloc[0]
BASE_URL = f"https://www.ebi.ac.uk:443/interpro/api/entry/pfam/IPR001368&annotation=alignment"

def output_list():
  #disable SSL verification to avoid config issues
  context = ssl._create_unverified_context()

  next = BASE_URL
  last_page = False

  
  #json header
  sys.stdout.write("{ \"results\": [\n")
  
  attempts = 0
  while next:
    try:
      req = request.Request(next, headers={"Accept": "application/json"})
      res = request.urlopen(req, context=context)
      # If the API times out due a long running query
      if res.status == 408:
        # wait just over a minute
        sleep(61)
        # then continue this loop with the same URL
        continue
      elif res.status == 204:
        #no data so leave loop
        break
      payload = json.loads(res.read().decode())
      next = payload["next"]
      attempts = 0
      if not next:
        last_page = True
    except HTTPError as e:
      if e.code == 408:
        sleep(61)
        continue
      else:
        # If there is a different HTTP error, it wil re-try 3 times before failing
        if attempts < 3:
          attempts += 1
          sleep(61)
          continue
        else:
          sys.stderr.write("LAST URL: " + next)
          raise e

    for i, item in enumerate(payload["results"]):
      
      sys.stdout.write(json.dumps(item))
      # for indented output replace the above line with the following
      # sys.stdout.write(json.dumps(item, indent=4))
      # for 1 record per line uncomment the following line
      # sys.stdout.write("\n")

      if last_page and i+1 == len(payload["results"]):
        sys.stdout.write("")
      else:
        sys.stdout.write(",\n")
      
    # Don't overload the server, give it time before asking for more
    if next:
      sleep(1)

  #json footer
  sys.stdout.write("\n] }\n")
  

if __name__ == "__main__":
  output_list()

### API: by sequence

In [104]:
sequence = df["meso_seq"].iloc[0]
url = f"https://www.ebi.ac.uk/interpro/api/entry/set?sequence={sequence}"
response = requests.get(url)
response.json()

{'entries': {'member_databases': {'pfam': 7765, 'cdd': 18582, 'pirsf': 82},
  'integrated': 11856,
  'unintegrated': 14522,
  'interpro': 0,
  'all': 26454},
 'sets': {'pfam': 655, 'cdd': 1246, 'pirsf': 44, 'all': 1945}}

### API: by sequence

In [99]:
sequence = df["meso_seq"].iloc[0]
url = f"https://www.ebi.ac.uk:443/interpro/api/entry/pfam?sequence={sequence}"
response = requests.get(url)
response.json()

{'count': 19632,
 'next': 'https://www.ebi.ac.uk/interpro/api/entry/pfam?cursor=cD1QRjAwMDIw&sequence=MLLSDRDLVSEIKSGDLSLEPFEPALLQPSSIDVRLDRFFRVFNNHLYTHIDPAEQQDDLTAEVEVTDGEAFVLHPGEFVLASTLEVITLGDQLAGRLEGKSSLGRLGLLTHSTAGFIDPGFSGHVTLELSNVANLPIKLWPGMKIGQLCIFRLSSPAEHPYGSAVYGSRYQGQRGPTPSRSAQNFRLWPTS',
 'previous': None,
 'results': [{'metadata': {'accession': 'PF00001',
    'name': '7 transmembrane receptor (rhodopsin family)',
    'source_database': 'pfam',
    'type': 'family',
    'integrated': 'IPR000276',
    'member_databases': None,
    'go_terms': None}},
  {'metadata': {'accession': 'PF00002',
    'name': '7 transmembrane receptor (Secretin family)',
    'source_database': 'pfam',
    'type': 'family',
    'integrated': 'IPR000832',
    'member_databases': None,
    'go_terms': None}},
  {'metadata': {'accession': 'PF00003',
    'name': '7 transmembrane sweet-taste receptor of 3 GCPR',
    'source_database': 'pfam',
    'type': 'domain',
    'integrated': 'IPR017978',
    'member_da

### API: by sequence

In [105]:
import requests

BASE_URL = "https://www.ebi.ac.uk:443/interpro/api/entry/all/pfam/?page_size=200"

# Define the protein sequences
meso_seq = "MLLSDRDLVSEIKSGDLSLEPFEPALLQPSSIDVRLDRFFRVFNNHLYTHIDPAEQQDDLTAEVEVTDGEAFVLHPGEFVLASTLEVITLGDQLAGRLEGKSSLGRLGLLTHSTAGFIDPGFSGHVTLELSNVANLPIKLWPGMKIGQLCIFRLSSPAEHPYGSAVYGSRYQGQRGPTPSRSAQNFRLWPTS"
thermo_seq = "MLLSDRDLRKELESGRLELDPFDPAMLQPSSIDVRLDRFFRVFDNTKYTHIDPSIQQDELTSLVEVGDGDEPFVLHPGEFVLGSTYEMVTLPDDLAGRLEGKSSLGRLGLLTHSTAGFIDPGFSGHITLELSNVANLPITLWPGMKIGQLCLFRLTSAAEHPYGSARAGSRYQHQRGPTPSRAYLNFSRIDTRR"

# Construct the query URLs
meso_url = f"{BASE_URL}&sequence={meso_seq}"
thermo_url = f"{BASE_URL}&sequence={thermo_seq}"

# Send the requests and parse the responses
meso_response = requests.get(meso_url).json()
thermo_response = requests.get(thermo_url).json()

# Extract the relevant information and print in a table format
print("SEQUENCE", "ACCESSION", "NAME", "SOURCE DATABASE", "MATCHES", sep="\t")
for result in meso_response["results"]:
    matches = result.get("matches", [])
    print(meso_seq[:20], result["metadata"]["accession"], result["metadata"]["name"], result["metadata"]["source_database"], matches, sep="\t")
for result in thermo_response["results"]:
    matches = result.get("matches", [])
    print(thermo_seq[:20], result["metadata"]["accession"], result["metadata"]["name"], result["metadata"]["source_database"], matches, sep="\t")


SEQUENCE	ACCESSION	NAME	SOURCE DATABASE	MATCHES
MLLSDRDLVSEIKSGDLSLE	PF00001	7 transmembrane receptor (rhodopsin family)	pfam	[]
MLLSDRDLVSEIKSGDLSLE	PF00002	7 transmembrane receptor (Secretin family)	pfam	[]
MLLSDRDLVSEIKSGDLSLE	PF00003	7 transmembrane sweet-taste receptor of 3 GCPR	pfam	[]
MLLSDRDLVSEIKSGDLSLE	PF00004	ATPase family associated with various cellular activities (AAA)	pfam	[]
MLLSDRDLVSEIKSGDLSLE	PF00005	ABC transporter	pfam	[]
MLLSDRDLVSEIKSGDLSLE	PF00006	ATP synthase alpha/beta family, nucleotide-binding domain	pfam	[]
MLLSDRDLVSEIKSGDLSLE	PF00007	Cystine-knot domain	pfam	[]
MLLSDRDLVSEIKSGDLSLE	PF00008	EGF-like domain	pfam	[]
MLLSDRDLVSEIKSGDLSLE	PF00009	Elongation factor Tu GTP binding domain	pfam	[]
MLLSDRDLVSEIKSGDLSLE	PF00010	Helix-loop-helix DNA-binding domain	pfam	[]
MLLSDRDLVSEIKSGDLSLE	PF00011	Hsp20/alpha crystallin family	pfam	[]
MLLSDRDLVSEIKSGDLSLE	PF00012	Hsp70 protein	pfam	[]
MLLSDRDLVSEIKSGDLSLE	PF00013	KH domain	pfam	[]
MLLSDRDLVSEIKSGDLSLE	PF00014	Kuni

### API: by sequence

In [106]:
import json
import sys
from urllib.error import HTTPError
from urllib.request import urlopen

def get_pfam_matches(seq):
    """
    Get all PFAM matches for a given sequence using InterPro API
    """
    api_url = "https://www.ebi.ac.uk/interpro/api"
    url = f"{api_url}/entry/all/pfam/?page_size=200&sequence={seq}"
    try:
        with urlopen(url) as res:
            data = json.loads(res.read().decode("utf-8"))
    except HTTPError:
        print(f"No PFAM matches found for sequence: {seq}")
        return []

    matches = []
    for result in data["results"]:
        accession = result["metadata"]["accession"]
        name = result["metadata"]["name"]
        source = result["metadata"]["source_database"]
        matches.append((accession, name, source))
    return matches


def main():
    meso_seq = "MSKEMKSLIVLLFL-AGLIASFAFSAEAGAKQSVKVDTEGTWTVPQAGKLQRFMMDLSR"
    thermo_seq = "MSKENKSLIVLLFL-AGLIASFAFSVEAGAKQSVKVDTEGTWTVPQAGKLQRFMMDLSR"

    print("Meso_seq Pfam matches:")
    meso_matches = get_pfam_matches(meso_seq)
    for match in meso_matches:
        print(match[0], match[1], match[2], sep="\t")

    print()
    print("Thermo_seq Pfam matches:")
    thermo_matches = get_pfam_matches(thermo_seq)
    for match in thermo_matches:
        print(match[0], match[1], match[2], sep="\t")

    # Use the fetch-protein-matches.py code with UniProt accession instead of seq
    # Example usage: python fetch-protein-matches.py P0A937

if __name__ == "__main__":
    main()

Meso_seq Pfam matches:
PF00001	7 transmembrane receptor (rhodopsin family)	pfam
PF00002	7 transmembrane receptor (Secretin family)	pfam
PF00003	7 transmembrane sweet-taste receptor of 3 GCPR	pfam
PF00004	ATPase family associated with various cellular activities (AAA)	pfam
PF00005	ABC transporter	pfam
PF00006	ATP synthase alpha/beta family, nucleotide-binding domain	pfam
PF00007	Cystine-knot domain	pfam
PF00008	EGF-like domain	pfam
PF00009	Elongation factor Tu GTP binding domain	pfam
PF00010	Helix-loop-helix DNA-binding domain	pfam
PF00011	Hsp20/alpha crystallin family	pfam
PF00012	Hsp70 protein	pfam
PF00013	KH domain	pfam
PF00014	Kunitz/Bovine pancreatic trypsin inhibitor domain	pfam
PF00015	Methyl-accepting chemotaxis protein (MCP) signalling domain	pfam
PF00016	Ribulose bisphosphate carboxylase large chain, catalytic domain	pfam
PF00017	SH2 domain	pfam
PF00018	SH3 domain	pfam
PF00019	Transforming growth factor beta like domain	pfam
PF00020	TNFR/NGFR cysteine-rich region	pfam
PF00021	

PF00001	7 transmembrane receptor (rhodopsin family)	pfam
PF00002	7 transmembrane receptor (Secretin family)	pfam
PF00003	7 transmembrane sweet-taste receptor of 3 GCPR	pfam
PF00004	ATPase family associated with various cellular activities (AAA)	pfam
PF00005	ABC transporter	pfam
PF00006	ATP synthase alpha/beta family, nucleotide-binding domain	pfam
PF00007	Cystine-knot domain	pfam
PF00008	EGF-like domain	pfam
PF00009	Elongation factor Tu GTP binding domain	pfam
PF00010	Helix-loop-helix DNA-binding domain	pfam
PF00011	Hsp20/alpha crystallin family	pfam
PF00012	Hsp70 protein	pfam
PF00013	KH domain	pfam
PF00014	Kunitz/Bovine pancreatic trypsin inhibitor domain	pfam
PF00015	Methyl-accepting chemotaxis protein (MCP) signalling domain	pfam
PF00016	Ribulose bisphosphate carboxylase large chain, catalytic domain	pfam
PF00017	SH2 domain	pfam
PF00018	SH3 domain	pfam
PF00019	Transforming growth factor beta like domain	pfam
PF00020	TNFR/NGFR cysteine-rich region	pfam
PF00021	u-PAR/Ly-6 domain	pfam
